In [11]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc

import keras as k
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split, KFold
from sklearn.preprocessing import MultiLabelBinarizer, MinMaxScaler
from sklearn.metrics import fbeta_score, precision_score, recall_score, make_scorer, average_precision_score
import matplotlib.pyplot as plt
import cv2
import warnings

%pylab inline
pylab.rcParams['figure.figsize'] = (15, 10)


import cv2
from tqdm import tqdm

Populating the interactive namespace from numpy and matplotlib


In [31]:
img_width, img_height = 64, 64
train_data_dir = "data/train"
validation_data_dir = "data/val"
nb_train_samples = 16
nb_validation_samples = 16
batch_size = 16
epochs = 1

# CNN specific preprocessing function, creating 4 dimensional input vectors
def preprocess_cnn(n_samples, rescaled_dim, f_path ="../data/train.csv"):
    df = pd.read_csv(f_path)
    df['split_tags'] = df['tags'].map(lambda row: row.split(" "))
    lb = MultiLabelBinarizer()
    y = lb.fit_transform(df['split_tags'])
    y = y[:n_samples]
    
    imgs = []
    # for each image, rescale to same dimension and flattern
    print "processing images..."
    count = 0
    for name in df.head(n_samples)['image_name'].values:
        raw_img = plt.imread('../data/train-jpg/{}.jpg'.format(name))
        #print raw_img
        imgs.append(cv2.resize(raw_img, (rescaled_dim, rescaled_dim), cv2.INTER_LINEAR)[:, :, :3])
        count+=1
        if count % 1000 == 0:
            print count, "processed"

    # remove dimenions, normalize  
    X = np.squeeze(np.array(imgs)) / 255.

    return np.array(X, np.float16), np.array(y, np.uint8), lb.classes_

def optimize_threshold(X_test, y_test, model):
    prediction = model.predict(X_test)
    true_label = y_test
    best_threshhold = [0.2]*17    
    for t in range(17):
        best_fbeta = 0
        temp_threshhold = [0.2]*17
        for i in range(100):
            temp_value = i / float(100)
            temp_threshhold[t] = temp_value
            temp_fbeta = fbeta_score(true_label, prediction > temp_threshhold, beta=2, average="samples")
            if temp_fbeta > best_fbeta:
                best_fbeta = temp_fbeta
                best_threshhold[t] = temp_value
    return best_threshhold


# Using a trained model, calculate the F2_score
# input: X_test, y_test, model
# output: a list of F2_score by class(length 17)
def calc_F2_score_cnn(X_test, y_test, model, thres = 0.2, avg_mode = None, opt = False):
    if opt:
        print "Optimizing threshold..."
        thres = optimize_threshold(X_test, y_test, model)
        print "Using optimized threshold: ", thres
    predicted = np.array(model.predict(X_test)) > thres
    score = fbeta_score(y_test, predicted, beta=2, average=avg_mode)
    return score

def calc_precision_cnn(X_test, y_test, model, thres = 0.2, avg_mode = None, opt = False):
    if opt:
        print "Optimizing threshold..."
        thres = optimize_threshold(X_test, y_test, model)
        print "Using optimized threshold: ", thres
    predicted = np.array(model.predict(X_test)) > thres
    score = precision_score(y_test, predicted, average=avg_mode)
    return score

def calc_recall_cnn(X_test, y_test, model, thres = 0.2, avg_mode = None, opt = False):
    if opt:
        print "Optimizing threshold..."
        thres = optimize_threshold(X_test, y_test, model)
        print "Using optimized threshold: ", thres
    predicted = np.array(model.predict(X_test)) > thres
    score = recall_score(y_test, predicted, average=avg_mode)
    return score

def print_results(scores, ind_to_classes):
    # print the scores with the classes
    results = [(ind_to_classes[l], scores[l]) for l in scores.argsort()[::-1]]
    print "************************RESULTS************************"
    for res in results:
        print res[0], res[1]
    print "************************END RESULTS************************"
    

In [34]:
model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))

# Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
for layer in model.layers:
    layer.trainable = False

#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(512, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(512, activation="relu")(x)
predictions = Dense(17, activation="sigmoid")(x)
model_final = Model(inputs = model.input, outputs = predictions)

model_final.compile(loss='binary_crossentropy', # We NEED binary here, since categorical_crossentropy l1 norms the output before calculating loss.
              optimizer='adam',
              metrics=['accuracy'])

In [35]:
NUM_TRAIN = 40482
NUM_SAMPLE = NUM_TRAIN
DIM = 64
X, y, ind_to_classes = preprocess_cnn(NUM_SAMPLE, DIM, "../data/train.csv")
print X.shape, y.shape

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.10, random_state=0)


processing images...
1000 processed
2000 processed
3000 processed
4000 processed
5000 processed
6000 processed
7000 processed
8000 processed
9000 processed
10000 processed
11000 processed
12000 processed
13000 processed
14000 processed
15000 processed
16000 processed
17000 processed
18000 processed
19000 processed
20000 processed
21000 processed
22000 processed
23000 processed
24000 processed
25000 processed
26000 processed
27000 processed
28000 processed
29000 processed
30000 processed
31000 processed
32000 processed
33000 processed
34000 processed
35000 processed
36000 processed
37000 processed
38000 processed
39000 processed
40000 processed
(40479, 64, 64, 3) (40479, 17)


In [36]:
model_final.fit(X_train, y_train,
          batch_size=64,
          epochs=5,
          verbose=1)
scores = calc_F2_score_cnn(X_valid, y_valid, model_final)

print_results(scores, ind_to_classes)

Epoch 1/5
36431/36431 [==============================] - 1607s - loss: 0.1863 - acc: 0.9263  
Epoch 2/5
36431/36431 [==============================] - 1555s - loss: 0.1657 - acc: 0.9350  
Epoch 3/5
36431/36431 [==============================] - 1571s - loss: 0.1614 - acc: 0.9368  
Epoch 4/5
36431/36431 [==============================] - 1556s - loss: 0.1593 - acc: 0.9372  
Epoch 5/5
36431/36431 [==============================] - 1488s - loss: 0.1573 - acc: 0.9381  
************************RESULTS************************
primary 0.988106367508
clear 0.956077630235
cloudy 0.813106796117
agriculture 0.808604287945
partly_cloudy 0.795201582983
road 0.73698515576
haze 0.718673218673
water 0.619625695498
habitation 0.545410860163
cultivation 0.493311036789
artisinal_mine 0.434782608696
bare_ground 0.028328611898
slash_burn 0.0
conventional_mine 0.0
selective_logging 0.0
blow_down 0.0
blooming 0.0
************************END RESULTS************************


In [37]:
#save model at a location
import datetime
def timestamp():
    return '{:%m%d_%H_%M_%s}'.format(datetime.datetime.now())

fn = "../data/cnn-3_" + timestamp() + '.h5'
model_final.save(fn)

In [39]:
calc_F2_score_cnn(X_valid, y_valid, model_final, thres = 0.2, avg_mode = "samples", opt=True)

Optimizing threshold...
Using optimized threshold:  [0.28, 0.17, 0.14, 0.08, 0.06, 0.26, 0.17, 0.07, 0.24, 0.25, 0.21, 0.2, 0.3, 0.2, 0.09, 0.05, 0.2]


/home/luda/cs231a-human-footprint/.env/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


0.86287071671215365

In [40]:
for idx, layer in enumerate(model_final.layers):
    print idx, layer.name 

0 input_7
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_conv4
11 block3_pool
12 block4_conv1
13 block4_conv2
14 block4_conv3
15 block4_conv4
16 block4_pool
17 block5_conv1
18 block5_conv2
19 block5_conv3
20 block5_conv4
21 block5_pool
22 flatten_7
23 dense_14
24 dropout_7
25 dense_15
26 dense_16


In [ ]:
from vis.utils import utils
from vis.visualization import visualize_activation

# The name of the layer we want to visualize
# (see model definition in vggnet.py)
layer_name = 'dense_16'
layer_idx = [idx for idx, layer in enumerate(model_final.layers) if layer.name == layer_name][0]

# Generate three different images of the same output index.
vis_images = []
for idx in range(17):
    img = visualize_activation(model_final, layer_idx, filter_indices=idx, max_iter=500)
    #img = utils.draw_text(img, str(idx))
    vis_images.append(img)

stitched = utils.stitch_images(vis_images)    
plt.axis('off')
plt.imshow(stitched)
plt.title(layer_name)
plt.show()

Working on filters: [0]
Working on filters: [1]